In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [31]:
base = pd.read_csv('/content/sample_data/testUpdated.csv', sep=',')

In [32]:
#visualizando a base, para escolher possiveis tratamentos
base.tail(2)

,Unnamed: 0,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
416,416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,0
417,417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C,0


In [33]:
#verificando quais atributos tem mais dados ausentes
base.isnull().sum()

,0
Unnamed: 0,0
PassengerId,0
Pclass,0
Name,0
Sex,0
Age,86
SibSp,0
Parch,0
Ticket,0
Fare,1


In [34]:
#removendo atributos desnecessarios ou muito ausentes(diminuindo dimensionalidade da base)
base = base.drop(base.columns[0], axis=1)
colunasRemovidas = ['PassengerId', 'Name', 'Ticket', 'Cabin']
base = base.drop(columns=colunasRemovidas)

In [35]:
#fazendo a media/moda dos valores ausentes para preencher os campos
base['Age'].fillna(base['Age'].mean(), inplace=True) #media
base['Fare'].fillna(base['Fare'].mean(), inplace=True)
base['Embarked'].fillna(base['Embarked'].mode()[0], inplace=True) #moda

In [36]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

#transformando atributos simbolicos para numericos

#primeiro, aqueles ordinais usando o LabelEncoder
#base_encoded = base.apply(LabelEncoder().fit_transform) -----para transformar todos atributos de uma vez
colunasLabelEnc = ['Sex', 'Embarked']
base[colunasLabelEnc] = base[colunasLabelEnc].apply(LabelEncoder().fit_transform)

#agora, os nao ordinais usando o OneHotEncoder
"""colunasOneHot = ['xxxxx']
base = ColumnTransformer(
    transformers=[('onehot', OneHotEncoder(), colunasOneHot)],
    remainder='passthrough'
 ).fit_transform(base)"""

#conferindo novamente
base.tail(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
416,3,1,30.27259,0,0,8.0500,2,0
417,3,1,30.27259,1,1,22.3583,0,0


In [37]:
import pickle
from imblearn.under_sampling import TomekLinks
from sklearn.model_selection import train_test_split
#separando os atributos da classe e salvando como pickle (pkl)
X_atributos = base.iloc[:, 0:7]
Y_classe = base.iloc[:, -1]

#balanceando a base de dados com undersampling
balanceamento_under = TomekLinks(sampling_strategy='auto')
X_under, Y_under = balanceamento_under.fit_resample(X_atributos, Y_classe)

#caso seja para dividir a base em treino e teste
"""X_treino, X_teste, Y_treino, Y_teste = train_test_split(X_under, Y_under, test_size = 0.20, random_state = 0)

with open('/content/sample_data/titanicTratadaBalanceada.pkl', mode = 'wb') as f:
  pickle.dump([X_treino, X_teste, Y_treino, Y_teste], f) """


#caso seja a base so de treino ou teste
with open('/content/sample_data/titanicTratadaBalanceada.pkl', mode = 'wb') as f:
  pickle.dump([X_under, Y_under], f)